Capstone - Part 5

Topic: Model Deployment

Created By: Jason

In [1]:
#Mounting our Drive to Google Colab
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [2]:
!pip install -q streamlit #restart runtime needed if unable to install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


In [4]:
! pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=b4a142c012bfc643c0ca31894017b5c5bd2bcb1586a2a46aaf4483e303eba4c3
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok


In [5]:
# import all libraries invovled
import streamlit as st # import streamlit for web development
import pickle
import numpy as np
import pandas as pd
from pyngrok import ngrok

Model Building - Artificial Neural Network (ANN)

In [6]:
#import data set
path = '/drive/MyDrive/Colab Notebooks/data_tour_cluster.csv'
data = pd.read_csv(path)

In [7]:
# the variable 'cluster' type is classified as 'numeric' - hence, changes needed to change
data_types_dict = {'Cluster': str} 
# change the data type 
data = data.astype(data_types_dict)

In [8]:
# Remove irrelavant columns 
data = data.drop(['Unnamed: 0'], axis=1)
data.dtypes

Cluster                    object
gender                     object
age                        object
household_income           object
education_level            object
occupation                 object
travel_purpose             object
pref_transport_mode        object
pref_length_stay           object
travel_pay_willingness     object
pref_poi                   object
pref_group_size            object
pref_travel_destination    object
dtype: object

In [9]:
# Label Encoding
from sklearn.preprocessing import LabelEncoder
# Label encode categorical variables
# gender # age # household income # education level # occupation

data_encoded = data.copy() # To serve as backup incase there is Pandas error
data_encoded[['gender', 'age', 'household_income','education_level', 'occupation', 'pref_transport_mode', 'travel_pay_willingness', 'pref_group_size', 'travel_purpose','pref_length_stay','pref_poi','pref_travel_destination']] = data_encoded[['gender', 'age', 'household_income','education_level', 'occupation', 'pref_transport_mode', 'travel_pay_willingness', 'pref_group_size','travel_purpose','pref_length_stay','pref_poi','pref_travel_destination']].apply(LabelEncoder().fit_transform)

In [10]:
# drop pref group size and education level
data_encoded = data_encoded.drop(['pref_group_size', 'education_level'], axis=1)

In [11]:
# Identify target variable 
Y = data_encoded["Cluster"]
X = data_encoded.drop("Cluster", axis = 1)

In [12]:
# Data Partitioning (70% train & 30% test set)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)  

(80, 10) (35, 10) (80,) (35,)


In [13]:
# ANN model was invovled as it is selected as the final model to be deployed
# Same architecture as developed previously (2 Hidden Layers, each with 100 neurons)
from sklearn.neural_network import MLPClassifier 
nnmodel = MLPClassifier(hidden_layer_sizes = [100,100], alpha = 5.0, random_state = 42, solver = 'lbfgs')

In [14]:
nnmodel.fit(X_train, y_train) #train the ANN

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=5.0, hidden_layer_sizes=[100, 100], random_state=42,
              solver='lbfgs')

In [15]:
y_pred = nnmodel.predict(X_test)
y_pred

array(['2', '2', '2', '1', '1', '1', '1', '2', '0', '2', '0', '1', '1',
       '1', '2', '0', '1', '1', '0', '1', '0', '2', '2', '1', '0', '1',
       '1', '2', '1', '0', '1', '0', '2', '2', '2'], dtype='<U1')

In [16]:
# check updated data frame
X_test.head()

,gender,age,household_income,occupation,travel_purpose,pref_transport_mode,pref_length_stay,travel_pay_willingness,pref_poi,pref_travel_destination
81,1,0,1,0,1,1,0,0,0,4
4,0,0,0,0,1,0,0,1,0,1
40,1,1,1,0,1,1,0,1,0,2
69,0,0,0,1,1,1,1,1,0,2
10,1,0,1,0,1,0,0,0,0,1


In [ ]:
# Save the developed model into a folder
import pickle
with open("NN_Model_pickle_2", "wb") as n:
    pickle.dump(nnmodel, n)

In [ ]:
NN_model_pickle_in = open("NN_Model_pickle_2", "rb")
NNmodel_deploy = pickle.load(NN_model_pickle_in)
with open("NN_Model_pickle_2", "rb") as n:
    NNmodel_deploy = pickle.load(n)

Creation of the content and body of the web application ( for prediction )

In [ ]:
%%writefile app.py
import streamlit as st
import pickle
import numpy as np
import pandas as pd

NN_model_pickle_in = open("NN_Model_pickle_2", "rb")
NNmodel_deploy = pickle.load(NN_model_pickle_in)
with open("NN_Model_pickle_2", "rb") as n:
    NNmodel_deploy = pickle.load(n)

def main():
  import numpy as np

  # Give title
  st.title('Travel Agency Operator Application')

  # Provide Content
  st.header('Web Application Detail')
  st.write('This application is strictly for tourist agent only :rotating_light:')
  st.write('The model implemented for classification is Artificial Neural Network')
  st.write('Kindly fill in the tourist detail for each section to compute the predictions.')
  st.write('The final result should show which cluster does this potential traveller falls under.')
  st.write('Kindly suggest the appropriate tour package to respective customers.')

  # Provide cluster details
  st.header('Cluster Category')
  st.write('At the current stage, there are only 3 clusters.')
  st.write('Cluster 0: Economic Family Package')
  st.write('Cluster 1: Holiday Adventure Package')
  st.write('Cluster 2: Premium Holiday Package')

  # Information to fill in
  st.header('Potential Traveller Information')
  st.write('Gender: Female = "0", Male = "1"')
  st.write('Age: 18 - 25 years old = "0", 26 years and above = "1"')
  st.write('Household Income: More than RM 4,851 = "0", Less than RM 4,850 = "1"')
  st.write('Occupation: Employed = "0", Student = "1"')
  st.write('Travel Purpose: Family Trip / Entertainment / Others = "0", Holiday & Leisure = "1"')
  st.write('Preferred Mode of Transport: Non_Flight = "0", Flight = "1"')
  st.write('Preferred Length Of Stay: Less than 3 days = "0", 3 Days = "1",4 days and above = "2"')
  st.write('Amount Willing to Pay for Travel: Less than RM 300 = "0", More than RM 300 = "1"')
  st.write('Preferred POI: Shopping Malls & Historical Site = "0", Beaches and waterfall = "1", Adventure and Activity = "2"')
  st.write('Preferred Travel Destination: Central Region = "0", East Coast Region  = "1", East Malaysia = "2", Northern Region = "3", Southern Region = "4"')


  # Receive inputs
  gender = st.text_input('Gender')
  age = st.text_input('Age')
  household_income = st.text_input('Household Income')
  occupation = st.text_input('Occupation')
  travel_purpose = st.text_input('Travel Purpose')
  pref_transport_mode = st.text_input('Preferred Mode of Transport')
  pref_length_stay = st.text_input('Preferred Length Of Stay')
  travel_pay_willingness = st.text_input('Amount Willing to Pay for Travel')
  pref_poi = st.text_input('Preferred POI')
  pref_travel_destination = st.text_input('Preferred Travel Destination')

  input_data = (gender,age,household_income,occupation,travel_purpose,pref_transport_mode,pref_length_stay,travel_pay_willingness,pref_poi,pref_travel_destination)
  input_data_as_numpy_array = np.asarray(input_data)
  input_data_reshaped = input_data_as_numpy_array.reshape(1,-1) 

  # code for prediction
  travel = ''

  # create button for prediction

  if st.button('Traveller Cluster Results'):
    travel = NNmodel_deploy.predict(input_data_reshaped) 

    if (travel[0] == 0):
        travel = 'This traveller is classified under cluster 0'
    elif (travel[0] == 1):
        travel = 'This traveller is classified under cluster 1'
    else:
        travel = 'This traveller is classified under cluster 2'

    st.success(travel)

if __name__ == '__main__':
  main()

Installation (and load) of PYNGROK to create the temporary web application

In [ ]:
ngrok.set_auth_token("NGROK_AUTH_TOKEN")
# https://dashboard.ngrok.com/cloud-edge/endpoints

In [ ]:
!nohup streamlit run app.py --server.port 80 & # to generate one external links
url = ngrok.connect(port = '80')
print(url)

In [ ]:
tunnels = ngrok.get_tunnels() # to generate multiple available external links
tunnels